<p float="left">
    <img src="https://earthdata.nasa.gov/img/earthdata-fb-image.jpg" width="300">
    
</p>

## **2021 ESIP Winter Meeting: Science in the Cloud Demos**

<!-- <img align="left" src="https://earthdata.nasa.gov/img/earthdata-fb-image.jpg" width="250"> -->

### Working with Cloud-Based NASA Earth Observations Data and Tools: 
#### AWS in-region access of NASA Earthdata products

This notebook provides a basic end-to-end workflow to interact with data "in-place" from the NASA Earthdata Cloud, by accessing AWS S3 locations provided by [NASA Harmony](http://harmony.earthdata.nasa.gov/) outputs without the need to download data. While these outputs can be downloaded locally, the cloud offers the ability to scale compute resources to perform analyses over large areas and time spans, which is critical as data volumes continue to grow. 

This workflow combines search, discovery, access, reformatting, basic analyses, and plotting components presented during the AGU 2020 Fall Meeting Workshop "Working with Cloud-Based NASA Earth Observations Data and Tools", with all materials available from https://github.com/podaac/AGU-2020. Though the example we're working with in this notebook only focuses on a small time and area to account for a large number of concurrent processing requests, this workflow can be modified and scaled up to suit a larger time range and region of interest. In the example used during AGU, this simplified workflow can be scaled up in order to better understand the relationship between river discharge and sea surface salinity for impact assessment. The Learning Objectives of the original tutorial are copied below:

#### Learning objectives:

- Understand the Pangeo BinderHub environment used during the workshop and how to execute code within a Jupyter Notebook
- Search for Liquid Water Equivalent (LWE) data from GRACE/GRACE-FO and Sea Surface Salinity (SSS) from SMAP 
- Execute programmatic data access queries, plotting, and direct in-region cloud access using open source Python libraries.
- Access data in Zarr format from Earthdata Cloud (AWS)
- Subset both, plot and compare coincident data.
- Identify resources, including the Earthdata Cloud Primer, for getting started with Amazon Web Services outside of the Workshop to access and work with data with a cloud environment.

___

<p float="left">
    <img src="https://github.com/pangeo-data/pangeo/raw/master/docs/_static/pangeo_simple_logo.svg" width="200">
    
</p>


### __Pangeo BinderHub__

First, some basics on the Pangeo compute environment used during the live workshop and how to interact with Jupyter Notebooks and the Jupyter Lab interface.

* [Pangeo BinderHub](https://binder.pangeo.io/): A multi-user server for interactive data analysis. This Hub is running in the AWS `us-west-2` region, which is where all Earthdata Cloud data and transformation service outputs are located. Pangeo is supported, in part, by the National Science Foundation (NSF) and the National Aeronautics and Space Administration (NASA). Google provided compute credits on Google Compute Engine. The Pangeo community promotes open, reproducible, and scalable science. We thank all at Pangeo for supporting the 2020 AGU Workshop.

See instructions at the bottom of this notebook for how to set up your own AWS EC2 instance so that you can perform the same cloud access within your personal AWS environment. 

___
### __Import modules__


In [1]:
import sys
!{sys.executable} -m pip install s3fs zarr xarray intake

from netrc import netrc
from platform import system
from getpass import getpass
from urllib import request
from http.cookiejar import CookieJar
from os.path import join, expanduser
from pprint import pprint
import intake
import s3fs
#import rasterio
import zarr
import matplotlib.pyplot as plt
import dask.array as da
import time
import requests
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.animation as animation
# import cartopy.crs as ccrs
# import cartopy
import s3fs
from IPython.display import HTML
import json


%matplotlib inline

## __Earthdata Login__

A NASA Earthdata Login account is required to access data, as well as discover restricted data, from the NASA Earthdata system. Please visit https://urs.earthdata.nasa.gov to register and manage your Earthdata Login account. This account is free to create and only takes a moment to set up.

At this point in time (as we are still transitioning to a cloud environment), in order to access data from the Earthdata Cloud, you need early access permissions.

In [2]:
TOKEN_DATA = ("<token>"
              "<username>%s</username>"
              "<password>%s</password>"
              "<client_id>PODAAC CMR Client</client_id>"
              "<user_ip_address>%s</user_ip_address>"
              "</token>")


def setup_cmr_token_auth(endpoint: str='cmr.earthdata.nasa.gov'):
    ip = requests.get("https://ipinfo.io/ip").text.strip()
    return requests.post(
        url="https://%s/legacy-services/rest/tokens" % endpoint,
        data=TOKEN_DATA % (input("Username: "), getpass("Password: "), ip),
        headers={'Content-Type': 'application/xml', 'Accept': 'application/json'}
    ).json()['token']['id']


def setup_earthdata_login_auth(endpoint: str='urs.earthdata.nasa.gov'):
    netrc_name = "_netrc" if system()=="Windows" else ".netrc"
    try:
        username, _, password = netrc(file=join(expanduser('~'), netrc_name)).authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        print('Please provide your Earthdata Login credentials for access.')
        print('Your info will only be passed to %s and will not be exposed in Jupyter.' % (endpoint))
        username = input('Username: ')
        password = getpass('Password: ')
    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)
    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)


# Get your authentication token for searching restricted records in the CMR:
_token = setup_cmr_token_auth(endpoint="cmr.earthdata.nasa.gov")

# Start authenticated session with URS to allow restricted data downloads:
setup_earthdata_login_auth(endpoint="urs.earthdata.nasa.gov")

Username:  amy.steiker
Password:  ·········


___
## __Data Search and Discovery__


[**JPL GRACE and GRACE-FO Mascon Ocean, Ice, and Hydrology Equivalent Water Height Coastal Resolution Improvement (CRI) Filtered Release 06 Version 02**](https://podaac.jpl.nasa.gov/dataset/TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06_V2)

Provides land water equivalent (LWE) thickness for observing seasonal changes in water storage around the river. When discharge is high, the change in water storage will increase, pointing to a wet season. This product provides gridded monthly global water storage/height anomalies in a single data file in netCDF format, and can be used for analysis for ocean, ice, and hydrology phenomena. Source data are from [GRACE](https://podaac.jpl.nasa.gov/GRACE) and [GRACE-FO](https://podaac.jpl.nasa.gov/GRACE-FO)

#### First, define the region of interest over the Amazon river basin and set the temporal range:

In [3]:
# Bounding Box spatial parameter in decimal degree 'W,S,E,N' format.
bounding_box = '-52,-2,-43,6'

# Each date in yyyy-MM-ddTHH:mm:ssZ format; date range in start,end format
temporal = '2020-11-01T00:00:00Z,2020-11-30T23:59:59Z'

#### Set up dictionary with the data product of interest

Before we search programmatically using the [Common Metadata Repository (CMR)](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html), we can use NASA Earthdata Search to visualize file coverage over multiple data sets and to access the same data you will be working with below: 
[Earthdata Search project](https://search.earthdata.nasa.gov/search/granules?p=C1938032626-POCLOUD&pg[0][gsk]=-start_date&q=C1938032626-POCLOUD&sb[0]=-52%2C-2%2C-43%2C6&m=1.1339994258337498!-51.837100966519046!6!1!0!0%2C2&qt=2020-11-01T00%3A00%3A00.000Z%2C2020-11-30T23%3A59%3A59.999Z&tl=1595962575!4!!)

In [6]:
search_parameters = { 
        'short_name': 'TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06_V2',
        'provider': 'POCLOUD',
        'bounding_box': bounding_box,
        'temporal': temporal,
        'token': _token,
}

#### Discover file number and file size 

Using CMR search, determine the number of files that exist over this time and area of interest, as well as the average size and total volume of those files.

In [7]:
search_url = "https://cmr.earthdata.nasa.gov/search/granules"
output_format="json"

parameters = {
        "scroll": "true",
        "page_size": 100,
}

response = requests.post(f"{search_url}.{output_format}", params=parameters, data=search_parameters)
response.raise_for_status()

hits = int(response.headers['CMR-Hits'])
if hits > 0:
    print(f"Found {hits} files")
    results = json.loads(response.content)
    granules = []
    granules.extend(results['feed']['entry'])
    granule_sizes = [float(granule['granule_size']) for granule in granules]
    print(f"The total size of all files is {sum(granule_sizes):.2f} MB")
else:
    print("Found no hits")

Found 3 files
The total size of all files is 2254.50 MB


#### Locate the data access URLs provided by the data product (collection) metadata:

GRACE data:

In [8]:
r = requests.get(url="https://cmr.earthdata.nasa.gov/search/granules.umm_json", 
                 params=search_parameters)
grace_gran = r.json()
print("files returned:",grace_gran['hits'])
grace_gran['items'][0]['umm']['RelatedUrls']

files returned: 3


[{'URL': 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-public/TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06_V2/GRCTellus.JPL.200204_202009.GLO.RL06M.MSCNv02CRI.cmr.json',
  'Type': 'EXTENDED METADATA',
  'Description': 'File to download'},
 {'URL': 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06_V2/GRCTellus.JPL.200204_202009.GLO.RL06M.MSCNv02CRI.nc',
  'Type': 'GET DATA',
  'Description': 'File to download'},
 {'URL': 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-public/TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06_V2/GRCTellus.JPL.200204_202009.GLO.RL06M.MSCNv02CRI.nc.md5',
  'Type': 'EXTENDED METADATA',
  'Description': 'File to download'}]

___
## __Data Access from NASA Earthdata Harmony__

There are several different methodologies to search and access data depending on your use case. The URLs above can be located with the data granule (file) metadata, and pulled into a list that can then be bulk downloaded as you've seen in the previous tutorials.

[Harmony](https://harmony.earthdata.nasa.gov/) is a growing effort across the NASA EOSDIS community to provide a consistent method to access and perform data subsetting and transformation services for data in the Earthdata Cloud. These services are processed in the cloud, with data archived in the cloud, and outputs can be accessed by downloading to a local machine or through direct in-region access via Amazon Web Services. These next steps walk through this access method to access the data outputs directly within the AWS `us-west-2` region so that we can read the data into memory within this AWS environment. 


**The Harmony source code is migrating to public Github repositories to enable community contributions and development. A Python library is under development to make these next steps faster and easier.**

#### __Get collection ID for Harmony__

Harmony operates on an input Collection concept-id (a CMR construct). Here's how you identify the CMR concept-id for the data products of interest:

In [11]:
r = requests.get(url="https://cmr.earthdata.nasa.gov/search/collections.umm_json", 
                 params=search_parameters)

grace_coll = r.json()
grace_coll['hits']
grace_coll_meta = grace_coll['items'][0]['meta']
grace_coll_id = grace_coll_meta['concept-id']
print('GRACE collection id:', grace_coll_id)

GRACE collection id: C1938032626-POCLOUD


#### __Request GRACE data reformatted to Zarr__:

[Zarr](https://zarr.readthedocs.io/en/stable/) is a format for the storage of chunked, compressed, N-dimensional arrays. Zarr also enables you to store arrays in memory, on disk, inside a Zip file, or on S3. Harmony's Zarr Reformatter service will transform the SMAP data from its native NetCDF format to Zarr, to allow us to open and download/read just the data that we require for our Amazon Basin study area.


For our science use case, ideally we'd also want to request an entire year but since this can take a long time to process with many concurrent requests during this live workshop, we will just request a month of data for demonstration purposes.

In [12]:
harmony_root = 'https://harmony.earthdata.nasa.gov'
harmony_params = {
    'collection_id': grace_coll_id,
    'ogc-api-coverages_version': '1.0.0',
    'variable': 'all',
    'lat':'(-2:6)',
    'lon':'(-52:-43)',
    'start': '2020-11-01T00:00:00Z',
    'stop':'2020-11-30T23:59:59Z',
    'format': 'application/x-zarr',
}

grace_url = harmony_root+'/{collection_id}/ogc-api-coverages/{ogc-api-coverages_version}/collections/{variable}/coverage/rangeset?format={format}&subset=lat{lat}&subset=lon{lon}&subset=time("{start}":"{stop}")'.format(**harmony_params)
print(grace_url)

https://harmony.earthdata.nasa.gov/C1938032626-POCLOUD/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?format=application/x-zarr&subset=lat(-2:6)&subset=lon(-52:-43)&subset=time("2019-01-01T00:00:00Z":"2019-12-31T23:59:59Z")


In [13]:
grace_response = request.urlopen(grace_url)
grace_results = grace_response.read()
grace_json = json.loads(grace_results)
print(json.dumps(grace_json, indent=2))
grace_jobId = grace_json['jobID']

{
  "username": "amy.steiker",
  "status": "running",
  "message": "Data in output files may extend outside the spatial bounds you requested.",
  "progress": 0,
  "createdAt": "2021-01-27T03:47:54.295Z",
  "updatedAt": "2021-01-27T03:47:54.295Z",
  "links": [
    {
      "title": "Job Status",
      "href": "https://harmony.earthdata.nasa.gov/jobs/737167e2-f8be-4fc9-9b69-5ce7f61affd1",
      "rel": "self",
      "type": "application/json"
    }
  ],
  "request": "https://harmony.earthdata.nasa.gov/C1938032626-POCLOUD/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?format=application%2Fx-zarr&subset=lat(-2%3A6)&subset=lon(-52%3A-43)&subset=time(%222019-01-01T00%3A00%3A00Z%22%3A%222019-12-31T23%3A59%3A59Z%22)",
  "jobID": "737167e2-f8be-4fc9-9b69-5ce7f61affd1"
}


In [14]:
grace_job_url = f'https://harmony.earthdata.nasa.gov/jobs/{grace_jobId}'

while True:
    loop_response = request.urlopen(grace_job_url)
    loop_results = loop_response.read()
    job_json = json.loads(loop_results)
    if job_json['status'] != 'running':
        break
    print(f"# Job status is running. Progress is {job_json['progress']} %. Trying again.")
    time.sleep(10)

grace_links = []
if job_json['status'] == 'successful' and job_json['progress'] == 100:
    print("# Job progress is 100%. Links to job outputs are displayed below:")
    grace_links = [link['href'] for link in job_json['links']]
    display(grace_links)

# Job status is running. Progress is 0 %. Trying again.
# Job status is running. Progress is 0 %. Trying again.
# Job status is running. Progress is 0 %. Trying again.
# Job status is running. Progress is 0 %. Trying again.
# Job status is running. Progress is 0 %. Trying again.
# Job status is running. Progress is 0 %. Trying again.
# Job status is running. Progress is 0 %. Trying again.
# Job status is running. Progress is 0 %. Trying again.
# Job status is running. Progress is 0 %. Trying again.
# Job status is running. Progress is 0 %. Trying again.
# Job status is running. Progress is 0 %. Trying again.
# Job status is running. Progress is 0 %. Trying again.
# Job progress is 100%. Links to job outputs are displayed below:


['https://harmony.earthdata.nasa.gov/jobs/737167e2-f8be-4fc9-9b69-5ce7f61affd1',
 'https://harmony.earthdata.nasa.gov/stac/737167e2-f8be-4fc9-9b69-5ce7f61affd1/',
 'https://harmony.earthdata.nasa.gov/cloud-access.sh',
 'https://harmony.earthdata.nasa.gov/cloud-access',
 's3://harmony-prod-staging/public/harmony/netcdf-to-zarr/aba03de6-ecc0-464e-9df2-70af686de807/',
 's3://harmony-prod-staging/public/harmony/netcdf-to-zarr/aba03de6-ecc0-464e-9df2-70af686de807/GRCTellus.JPL.200204_202009.GLO.RL06M.MSCNv02CRI.zarr',
 's3://harmony-prod-staging/public/harmony/netcdf-to-zarr/aba03de6-ecc0-464e-9df2-70af686de807/GRCTellus.JPL.200204_202010.GLO.RL06M.MSCNv02CRI.zarr',
 's3://harmony-prod-staging/public/harmony/netcdf-to-zarr/aba03de6-ecc0-464e-9df2-70af686de807/GRCTellus.JPL.200204_202011.GLO.RL06M.MSCNv02CRI.zarr']

#### __Access urls for the output zarr files__

The zarr dataset is staged for us in an S3 bucket. The url is the last one in the list shown above.

Select the url and display below:

In [15]:
grace_zarr_urls = grace_links[-1]
grace_zarr_urls

's3://harmony-prod-staging/public/harmony/netcdf-to-zarr/aba03de6-ecc0-464e-9df2-70af686de807/GRCTellus.JPL.200204_202011.GLO.RL06M.MSCNv02CRI.zarr'

**Access credentials for the output zarr file**

Credentials provided in the Harmony job response provide authenticated access to your staged S3 resources.

Grab the credentials as a JSON string, load to a Python dictionary, and display their expiration date:

In [16]:
with request.urlopen(f"https://harmony.earthdata.nasa.gov/cloud-access") as f:
    creds = json.loads(f.read())

creds['Expiration']

'2021-01-27T11:54:12.000Z'

### Open staged zarr files with *s3fs*

We use the AWS `s3fs` package to get metadata about the zarr data store and read in the credentials we pulled from our Harmony job response:

In [ ]:
zarr_fs = s3fs.S3FileSystem(
    key=creds['AccessKeyId'],
    secret=creds['SecretAccessKey'],
    token=creds['SessionToken'],
    client_kwargs={'region_name':'us-west-2'},
)

#### GRACE data loaded into xarray

 `xarray` is a python package designed to work with multi-dimensional arrays.  See the [xarray website](http://xarray.pydata.org/en/stable/) for more information. This next code block will take all of the month's worth of SMAP data we pulled above from Harmony into xarray in a single command.

In [ ]:
grace_zarr_store = zarr_fs.get_mapper(root=grace_zarr_urls, check=False)
grace_zarr_dataset = zarr.open(grace_zarr_store)

print(grace_zarr_dataset.tree())

In [ ]:
print(grace_zarr_dataset.lwe_thickness.info)

#### __Open staged zarr file with *xarray*__

Read more about `xarray`'s zarr reader here: http://xarray.pydata.org/en/stable/generated/xarray.open_zarr.html

This xarray method allows you to pull in the Zarr outputs from Harmony directly into an xarray dataset. 

Open the zarr dataset and print the dataset "header":

In [ ]:
ds_GRACE = xr.open_zarr(grace_zarr_store)
print(ds_GRACE)

**Subset by Latitude/Longitude**

Once we have obtained all the data, to make processing quicker, we are going to subset datasets by latitude/longitude for the Amazon River estuary.

Once we have obtained the GRACE-FO data, we should spatial subset the data to the minimal area covering the Amazon River estuary. This will reduce processing load and reduce cloud costs for the user.

Make a GRACE-FO subset and display the min, max of the *lat* and *lon* variables:

In [ ]:
subset_GRACE = ds_GRACE.sel(lat=slice(-18, 10), lon=slice(275, 330))
print(subset_GRACE.lat.min().data, 
      subset_GRACE.lat.max().data,
      subset_GRACE.lon.min().data,
      subset_GRACE.lon.max().data)

**Select the variable for Land Water Equivalent Thickness (*lwe_thickness*)**

Grab the land water equivalent thickness variable from the GRACE subset:

In [ ]:
lwe = subset_GRACE.lwe_thickness
print(lwe)

___

## __Plotting and analysis__

__Plot and animate the GRACE data over time__

Now that we have a time/space slice of interest from the GRACE collection, let's plot the first time step, to see what we've got (or to see what it looks like). First we'll define two functions to make the plotting (and next animation step) a bit more convenient:

In [ ]:
def setup_map(ax, pmap, ds_subset, x, y, var, t, cmap, levels, extent):
    title = str(pd.to_datetime(ds_subset.time[t].values))
    pmap.set_title(title, fontsize=14)
    pmap.coastlines()
    pmap.set_extent(extent)
    pmap.add_feature(cartopy.feature.RIVERS)
    variable_desired = var[t,:,:]
    cont = pmap.contourf(x, y, variable_desired, cmap=cmap, levels=levels, zorder=1)
    return cont

def animate_ts(framenumber, ax, pmap, ds_subset, x, y, var, t, cmap, levels, extent):
    cont = setup_map(ax, pmap, ds_subset, x, y, var, t + framenumber, cmap, levels, extent) 
    return cont

In [ ]:
# Initialize a matplotlib plot object and add subplot:
fig = plt.figure(figsize=[13,9]) 
ax = fig.add_subplot(1, 1, 1)

# Configure axes to display projected data using PlateCarree crs:
pmap = plt.axes(projection=ccrs.PlateCarree())

# Get arrays of x and y to label the plot axes:
x,y = np.meshgrid(subset_GRACE.lon, subset_GRACE.lat)                        

# Set a few constants for plotting the GRACE-FO time series:
time_start  = 168
cmap_name   = "bwr_r"
cmap_levels = np.linspace(-100., 100., 14)
map_extent  = [-85, -30, -16, 11]

# Plot the first timestep: 
cont = setup_map(ax, pmap, subset_GRACE, x, y, lwe, time_start, cmap_name, cmap_levels, map_extent)

fig.colorbar(cont, ticks=cmap_levels, orientation='horizontal', label='Land Water Equivalent Thickness (cm)')

#### Animate changes over time

Let's now explore how land water mass changes throughout the year 2019, by creating an animation of GRACE monthly land water equivalent (LWE) maps over the Amazon River.

Plot all the 2019 timesteps sequentially to create an animation of land water equivalent thickness for the Amazon Rainforest territories:

In [ ]:
ani = animation.FuncAnimation(fig, animate_ts, frames=range(0,12), fargs=(
    ax, pmap, subset_GRACE, x, y, lwe, time_start, cmap_name, cmap_levels, map_extent
), interval=500)

HTML(ani.to_html5_video())

User note: You will need to install 'ffmpeg' in the cmd prompt to save the .mpg to disk. Use the following command to install from the conda-forge channel:

```shell
conda install -c conda-forge ffmpeg
```

Uncomment, run the next cell to save the animation to MP4:

In [ ]:
#ani.save("earthdatacloud_animation_GRACEFO.mp4", writer=animation.FFMpegWriter())

## __Tutorial Summary and Additional Resources__

The building blocks are now in place to do a longer time series analysis across GRACE and SMAP data, to better understand the relationship between river discharge and sea surface salinity for impact assessment. 

To conclude, we've searched programmatically for data archived in the PO.DAAC Earthdata Cloud over a region and time period of interest, requested the data from the Harmony API, read the data directly into `xarray` from the staged s3 location within AWS `us-west-2` without having to pull the data down into local storage, and performed subsetting and plotting in preparation for a time series analysis. 

____

## __Set up for in-region access__


__This notebook must be running within an AWS EC2 instance running in the `us-west-2` region.__

The Pangeo BinderHub instance provided through the Github repository already takes care of steps 1 and 2, but these instructions are provided so that you can set this up in your own AWS account outside of the provided cloud environment.

1. Follow tutorials 01 through 03 of the [NASA Earthdata Cloud Primer](https://earthdata.nasa.gov/learn/user-resources/webinars-and-tutorials/cloud-primer) to set up an EC2 instance within us-west-2. Ensure you are also following step 3 in the ["Jupyter Notebooks on AWS EC2 in 12 (mostly easy) steps"](https://medium.com/@alexjsanchez/python-3-notebooks-on-aws-ec2-in-15-mostly-easy-steps-2ec5e662c6c6) article to set the correct security group settings needed to connect your local port to your EC2’s notebook port thru SSH.

2. Follow the remaining instructions in the Medium article above up until Step 11 (running Jupyter Lab). These instructions include installation of Anaconda3 (including Jupyter Lab) in your ec2 instance. Note the following updates and suggestions:
    * Step 5: Type the following command instead of what is suggested in the article: `ssh -i "tutorialexample.pem" ec2-user@ec2-54-144-47-199.compute-1.amazonaws.com -L 9999:localhost:8888`. This will eliminate the need to create a ssh config file in your home directory (Step 10).
    * As of December 2020, the most current Anaconda3 Linux distribution is: https://repo.anaconda.com/archive/Anaconda3-2020.11-Linux-x86_64.sh
    * The Anaconda installation prompts are not the same as in the article. You will not be prompted to include Anaconda3 in your .bashrc PATH so you can skip to their step 9. Instead select "yes" to initialize Anaconda by running `conda init`. 
    * For this notebook in particular, the following package installations are needed:
    
    `conda install -c conda-forge cartopy`
    
    `conda install -c conda-forge ffmpeg`
    
    `pip install s3fs zarr xarray pystac intake`
    
    * In order to clone the notebook within Jupyter Lab, git is needed:
    
    `sudo yum install git`
Before moving over to Jupyter Lab, set up your Earthdata Login authentication and Harmony access keys:

3. Setup your `~/.netrc` for Earthdata Login in your ec2 instance:

```
cd ~
touch .netrc
echo "machine urs.earthdata.nasa.gov login uid_goes_here password password_goes_here" > .netrc
chmod 0600 .netrc
```

4. Run the following in your ec2 instance terminal window to generate short-term Harmony access keys:

`curl -Ln -bj https://harmony.earthdata.nasa.gov/cloud-access.sh`

5. Set your environment variables based on the keys provided in step 4:

`export AWS_ACCESS_KEY_ID='...
export AWS_SECRET_ACCESS_KEY='...'
export AWS_SESSION_TOKEN='...'
export AWS_DEFAULT_REGION='us-west-2'`

Note that these expire within 8 hours of the script generation.

6. Launch jupyter lab:

`jupyter lab --no-browser`

Copy the URL that begins with `http://localhost:8888` into a browser window. Replace `8888` with `9999`. 

You should now be up and running with JupyterLab in your EC2!

7. Clone this repository

Open a Terminal in the Jupyter Lab launcher and enter the following git clone command:

`git clone https://github.com/podaac/AGU-2020.git`

The repository, including all notebooks, are now cloned and can be opened within Jupyter Lab.
****